In [ ]:
import os
import requests
import json
import logging
from pdf2image import convert_from_path
import numpy as np
import pytesseract
import re
import spacy
from spacy.matcher import Matcher
import torch
import transformers
import peft
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from pathlib import Path
from awq import AutoAWQForCausalLM
import language_tool_python
from faker import Faker
fake = Faker('ru_RU')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['HF_TOKEN'] = 'ТОКЕН'

In [ ]:
# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

In [ ]:
# Функция для извлечения текста из PDF-файлов
def extract_data(pdf_path):
    extracted_text = ""
    pages = convert_from_path(pdf_path, dpi=300)
    for page in pages:
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(np.array(page), lang='rus', config=custom_config)
        extracted_text += text
    return extracted_text

In [ ]:
# Функция для извлечения адреса электронной почты
def extract_email(text):
    """
    Извлекает первый найденный адрес электронной почты из предоставленного текста.

    Параметры:
    - text (str): Текст, из которого нужно извлечь адрес электронной почты.

    Возвращает:
    - str или None: Извлеченный адрес электронной почты, если найден, иначе None.
    """
    pattern = r"[\w\.-]+@[\w\.-]+"
    match = re.search(pattern, text)
    return match.group() if match else None

# Функция для извлечения номера телефона
def extract_phone_number(text):
    """
    Извлекает первый найденный номер телефона из предоставленного текста.

    Параметры:
    - text (str): Текст, из которого нужно извлечь номер телефона.

    Возвращает:
    - str или None: Извлеченный номер телефона, если найден, иначе None.
    """
    pattern = r"(\+7|8)?\s*\(?\d{3}\)?\s*\d{3}[-\s]?\d{2}[-\s]?\d{2}"
    match = re.search(pattern, text)
    return match.group() if match else None

# Функция для извлечения имени
def extract_name(text):
    """
    Извлекает первое найденное имя из предоставленного текста, предполагая, что оно написано заглавными буквами.

    Параметры:
    - text (str): Текст, из которого нужно извлечь имя.

    Возвращает:
    - str или None: Извлеченное имя, если найдено, иначе None.
    """
    pattern = r"^\s*([А-ЯЁ]+\s+[А-ЯЁ]+)"
    match = re.search(pattern, text, re.MULTILINE)
    return match.group(1) if match else None

# Функция для извлечения ссылки на портфолио
def extract_portfolio(text):
    """
    Извлекает первый найденный URL портфолио из предоставленного текста.

    Параметры:
    - text (str): Текст, из которого нужно извлечь URL.

    Возвращает:
    - str или None: Извлеченный URL, если найден, иначе None.
    """
    pattern = r"https?://[^\s]+"  # Поиск URL
    match = re.search(pattern, text)
    return match.group() if match else None

# Функция для обработки текста с помощью SpaCy
def get_nlp_doc(text):
    """
    Обрабатывает предоставленный текст с помощью SpaCy для создания объекта документа SpaCy.

    Параметры:
    - text (str): Текст для обработки.

    Возвращает:
    - Doc: Объект документа SpaCy.
    """
    nlp = spacy.load("ru_core_news_sm")
    return nlp(text)

# Функция для извлечения категорий из резюме
def extract_categories(text):
    """
    Извлекает и классифицирует различные разделы резюме на основе предопределенных ключевых слов, используя Matcher SpaCy.

    Параметры:
    - text (str): Текст резюме.

    Возвращает:
    - dict: Словарь с ключами как категориями и значениями как извлеченный текст для каждой категории.
    """
    nlp = spacy.load("ru_core_news_sm")

    matcher = Matcher(nlp.vocab)

    # Определение паттернов для каждой категории
    patterns = {
        "Образование": [[{"LOWER": "образование"}]],
        "Опыт": [[{"LOWER": "опыт"}]],
        "Навыки": [[{"LOWER": "навыки"}]],
        "Ключевые достижения": [[{"LOWER": "ключевые"}, {"LOWER": "достижения"}]],
        "Личное заявление": [[{"LOWER": "личное"}, {"LOWER": "заявление"}]],
    }

    # Добавление паттернов в matcher
    for category, pattern in patterns.items():
        matcher.add(category, pattern)

    doc = get_nlp_doc(text)
    matches = matcher(doc)
    categories = {category: "" for category in patterns.keys()}
    start_pos = {category: None for category in patterns.keys()}

    for match_id, start, end in matches:
        rule_id = nlp.vocab.strings[match_id]
        start_pos[rule_id] = start

    for category, pos in start_pos.items():
        if pos is not None:
            end_pos = len(doc)
            for other_cat, other_pos in start_pos.items():
                if other_pos is not None and other_pos > pos:
                    end_pos = min(end_pos, other_pos)
            span = doc[pos:end_pos]
            categories[category] = span.text.strip()

    return categories

# Функция для извлечения опыта работы
def extract_experiences(doc):
    """
    Извлекает подробный опыт работы из раздела 'Опыт' резюме.

    Параметры:
    - doc (Doc): Объект документа SpaCy, содержащий текст раздела 'Опыт'.

    Возвращает:
    - list: Список словарей, каждый из которых представляет собой извлеченный опыт работы.
    """
    experiences = []
    index = -1
    for sent in doc.sents:
        labels = [ent.label_ for ent in sent.ents]

        if "DATE" in labels and "ORG" in labels and "GPE" in labels:
            experiences.append(
                {
                    "дата": "",
                    "место": "",
                    "название организации": "",
                    "описание организации": "",
                    "должность": "",
                    "описание": "",
                }
            )

            index += 1

            date = [ent.text for ent in sent.ents if ent.label_ == "DATE"]
            experiences[index]["дата"] = date[0] if date else ""

            org = [ent.text for ent in sent.ents if ent.label_ == "ORG"]
            experiences[index]["название организации"] = org[0] if org else ""

            place = [ent.text for ent in sent.ents if ent.label_ == "GPE"]
            experiences[index]["место"] = place[0] if place else ""

            sent_split = sent.text.split("\n")
            sent_split = [s for s in sent_split if s.strip() and "опыт" not in s.lower()]
            role = sent_split[0].split(",")[0] if len(sent_split) > 0 else ""
            experiences[index]["должность"] = role.replace(date[0], "").strip()

            org_description = sent_split[1] if len(sent_split) > 1 else ""
            experiences[index]["описание организации"] = org_description.strip()

        else:
            if index >= 0:
                experiences[index]["описание"] += sent.text.replace("\n", "") + " "
            else:
                print("Дата, организация или место не найдены в предложении")

    return experiences

# Функция для извлечения образования
def extract_education(doc):
    """
     Извлекает подробную историю образования из раздела 'Образование' резюме.

    Параметры:
    - doc (Doc): Объект документа SpaCy, содержащий текст раздела 'Образование'.

    Возвращает:
    - list: Список словарей, каждый из которых представляет собой извлеченную историю образования.
    """
    education = []
    index = -1
    for sent in doc.sents:
        labels = [ent.label_ for ent in sent.ents]

        if "DATE" in labels and "ORG" in labels and "GPE" in labels:
            education.append(
                {
                    "дата": "",
                    "место": "",
                    "учебное заведение": "",
                    "название образования": "",
                    "описание": "",
                }
            )

            index += 1

            date = [ent.text for ent in sent.ents if ent.label_ == "DATE"]
            education[index]["дата"] = date[0] if date else ""

            org = [ent.text for ent in sent.ents if ent.label_ == "ORG"]
            education[index]["учебное заведение"] = org[0] if len(org) > 0 else ""

            place = [ent.text for ent in sent.ents if ent.label_ == "GPE"]
            education[index]["место"] = place[0] if place else ""

            sent_split = sent.text.split("\n")
            sent_split = [s for s in sent_split if s.strip() and "образование" not in s.lower()]
            formation_name = sent_split[0].split(",")[0] if len(sent_split) > 0 else ""
            education[index]["название образования"] = formation_name.strip()

            description = ""
            for i, word in enumerate(sent.text.split()):
                if word.lower() == "модули:":
                    description = " ".join(sent.text.split()[i + 1:])
                    break
            education[index]["описание"] = "Модули: " + description.strip() + " "

        else:
            if index >= 0:
                education[index]["описание"] += sent.text.replace("\n", "") + " "
            else:
                print("Дата, организация или место не найдены в предложении")

    return education

# Функция для извлечения данных резюме
def extract_resume_data(text):
    """
    Интегрирует все функции извлечения, чтобы структурировать неструктурированный текст резюме в категоризированные данные.

    Параметры:
    - text (str): Полный текст резюме.

    Возвращает:
    - dict: Словарь со структурированными данными резюме, категоризированными по образованию, опыту и т.д.
    """
    email = extract_email(text)
    phone_number = extract_phone_number(text)
    name = extract_name(text)
    portfolio = extract_portfolio(text)
    categories = extract_categories(text)

    # Генерация поддельных данных
    fake_name = fake.name()
    fake_email = fake.email()
    fake_phone_number = fake.phone_number()

    education_data = extract_education(categories)
    experience_data = extract_experiences(categories)

    data = {
        "образование": f"Имя: {fake_name}\nОбразование: {education_data}",
        "опыт": f"Имя: {fake_name}\nОпыт: {experience_data}",
        "навыки": f"Имя: {fake_name}\nНавыки: {categories.get('Навыки', '')}",
        "ключевые_достижения": f"Имя: {fake_name}\nКлючевые достижения: {categories.get('Ключевые достижения', '')}",
        "личное_заявление": f"Имя: {fake_name}\nЛичное заявление: {categories.get('Личное заявление', '')}",
        "контактная_информация": f"Имя: {fake_name}\nEmail: {fake_email}\nТелефон: {fake_phone_number}\nПортфолио: {portfolio}",
    }

    return data

In [ ]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"

# Загрузка конфигурации PEFT
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загрузка основной модели
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"  # "auto" для автоматического распределения по доступным GPU
)

# Загрузка адаптера
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)

# Перевод модели в режим оценки
model.eval()

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

# Настройка конфигурации генерации
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

In [ ]:
# Функция для проверки наличия ошибок
def check_error(resume_text, instruction):
    if instruction.startswith("Проверь, есть ли фотография"):
        return "фото" not in resume_text.lower()
    elif instruction.startswith("Проверь, указаны ли контактные данные"):
        return not (extract_email(resume_text) or extract_phone_number(resume_text))
    elif instruction.startswith("Проверь, описан ли опыт работы"):
        return "опыт работы" not in resume_text.lower() or "нет" in resume_text.lower()
    elif instruction.startswith("Проверь текст на наличие орфографических, речевых и пунктуационных ошибок"):
        return "ошибка" in resume_text.lower()
    return False

# Инструкции для выявления ошибок и генерации рекомендаций
error_detection_instructions = {
    "отсутствует_фото": "Проверь, есть ли фотография в резюме. Если фото отсутствует, добавь ошибку 'Отсутствует фотография'.",
    "отсутствует_контактная_информация": "Проверь, указаны ли контактные данные (email, телефон). Если нет, добавь ошибку 'Отсутствует контактная информация'.",
    "краткий_опыт": "Проверь, описан ли опыт работы подробно. Если опыт не описан или указан как 'нет', добавь ошибку 'Опыт работы не описан подробно'.",
    "ошибки_грамматики": "Проверь текст на наличие орфографических ошибок. Если ошибки найдены, добавьте их в список ошибок."
}

recommendation_instructions = {
    "отсутствует_фото": "Если в резюме отсутствует фотография, предложи рекомендацию: 'Добавьте фотографию'.",
    "отсутствует_контактная_информация": "Если отсутствуют контактные данные, предложи рекомендацию: 'Укажите email и телефон'.",
    "краткий_опыт": "Если опыт работы не описан подробно, предложи рекомендацию: 'Опишите опыт работы более детально, даже если он отсутствует'.",
    "ошибки_грамматики": "Если найдены орфографические ошибки, предложи рекомендацию: 'Исправьте орфографические ошибки'."
}

# Функция для проверки грамматики
def check_grammar(text):
    tool = language_tool_python.LanguageTool('ru-RU')
    matches = tool.check(text)
    return matches

def generate_errors_and_recommendations(resume_text):
    errors = []
    recommendations = []

    # Проверка грамматики
    grammar_errors = check_grammar(resume_text)
    if grammar_errors:
        errors.append("ошибки_грамматики")
        recommendations.append("Исправьте грамматические ошибки.")

    # Остальные проверки
    for error_type, instruction in error_detection_instructions.items():
        if check_error(resume_text, instruction):
            errors.append(error_type)

    for error_type in errors:
        recommendation = recommendation_instructions.get(error_type, "Неизвестная ошибка")
        recommendations.append(recommendation)

    return {
        "резюме_текст": resume_text,
        "ошибки": errors,
        "рекомендации": recommendations
    }

def generate_training_data_jsonl(pdf_path, output_file, num_samples=1000):
    try:
        logging.info("Создание выходной директории, если она не существует.")
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        logging.info("Извлечение текста резюме из PDF.")
        resume_text = extract_data(pdf_path)
        if not resume_text:
            logging.warning("Текст не извлечен из PDF")
            return

        logging.info("Генерация ошибок и рекомендаций.")
        result = generate_errors_and_recommendations(resume_text)

        # Сохранение результата в JSON-файл
        with open(output_file, "w", encoding='utf-8') as json_file:
            json.dump(result, json_file, ensure_ascii=False, indent=4)

        logging.info("Файл обучающих данных в формате JSON успешно сгенерирован.")

    except Exception as e:
        logging.error(f"Ошибка при обработке PDF или генерации данных: {e}")

In [ ]:
import zipfile

zip_path = "/content/resume_data_pdf.zip"
extract_path = "/content/resume_data_pdf"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Функция для обработки всех PDF-файлов в папке
def process_pdf_folder(folder_path, output_file):
    all_results = []

    # Создание директории
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Перебор всех файлов в указанной директории
    for pdf_file in Path(folder_path).glob("*.pdf"):
        logging.info(f"Обработка файла: {pdf_file.name}")
        resume_text = extract_data(pdf_file)

        if not resume_text:
            logging.warning(f"Текст не извлечен из файла {pdf_file.name}")
            continue

        # Генерация ошибок и рекомендаций
        result = generate_errors_and_recommendations(resume_text)
        all_results.append(result)

    # Сохранение всех результатов в один JSON-файл
    with open(output_file, "w", encoding='utf-8') as json_file:
        json.dump(all_results, json_file, ensure_ascii=False, indent=4)

    logging.info(f"Все результаты успешно сохранены в {output_file}")

# Примеры использования
folder_path = "/content/resume_data_pdf.zip"
output_file = "train_data/training_data.json"  # Имя выходного файла
process_pdf_folder(folder_path, output_file)

# Пример использования для одного файла
generate_training_data_jsonl(
    "/content/resume_data_pdf/Khrustalev_Sergei_774_tekhnicheskii_774_direktor (1).pdf",
    "train_data/training_data.jsonl",
    200,
)